In [8]:
import numpy as np
import pandas as pd
from catboost import CatBoost

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
for i in train['Name']:
    if 'Master.' in i:
        train['Name'] = train['Name'].replace({i:int(0)})
    elif 'Miss.' in i:
        train['Name'] = train['Name'].replace({i:int(1)})
    elif 'Mr.' in i:
        train['Name'] = train['Name'].replace({i:int(2)})
    elif 'Mrs.' in i:
        train['Name'] = train['Name'].replace({i:int(3)})
    
train['Name'] = train['Name'].astype(int)

In [11]:
for i in test['Name']:
    if 'Master.' in i:
        test['Name'] = test['Name'].replace({i:int(0)})
    elif 'Miss.' in i:
        test['Name'] = test['Name'].replace({i:int(1)})
    elif 'Mr.' in i:
        test['Name'] = test['Name'].replace({i:int(2)})
    elif 'Mrs.' in i:
        test['Name'] = test['Name'].replace({i:int(3)})
test['Name'] = test['Name'].astype(int)
# test['Name'] = pd.to_numeric(test['Name'],errors='coerce').astype(int)

In [12]:
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Sex'] = train['Sex'].map({'male':1,'female':0})
train['Embarked'] = train['Embarked'].map({'S':3,'C':2,'Q':1})
# train.drop(['PassengerId','Name','Ticket','Cabin'],axis="columns",inplace=True)
train['Family'] = train['SibSp']+train['Parch']
train.drop(['PassengerId','Ticket','Cabin','SibSp','Parch'],axis="columns",inplace=True)

In [13]:
test['Age'] = test['Age'].fillna(test['Age'].mean())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Sex'] = test['Sex'].map({'male':1,'female':0})
test['Embarked'] = test['Embarked'].map({'S':3,'C':2,'Q':1})
# test.drop(['Name','Ticket','Cabin'],axis="columns",inplace=True)
test['Family'] = test['SibSp']+test['Parch']
test.drop(['Ticket','Cabin','SibSp','Parch'],axis="columns",inplace=True)

In [14]:
X_train = train.iloc[:,1:]
Y_train = train.iloc[:,0]
X_test = test.iloc[:,1:]

In [15]:
X_train.head()

,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,3,2,1,22.0,7.2500,3,1
1,1,3,0,38.0,71.2833,2,1
2,3,1,0,26.0,7.9250,3,0
3,1,3,0,35.0,53.1000,3,1
4,3,2,1,35.0,8.0500,3,0


In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ct = ColumnTransformer([('xyz',StandardScaler(),[3,4])], remainder='passthrough')
X_train = ct.fit_transform(X_train)
ct = ColumnTransformer([('xyz',StandardScaler(),[3,4])], remainder='passthrough')
X_test = ct.fit_transform(X_test)

In [17]:
from sklearn.model_selection import train_test_split
data_tr, data_te, result_tr, result_te = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

In [18]:
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [19]:
model = CatBoost()
grid_search_result = model.grid_search(grid, 
                                       X=data_tr, 
                                       y=result_tr, 
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6125628	test: 0.5926652	best: 0.5926652 (0)	total: 150ms	remaining: 2m 29s
1:	learn: 0.6017578	test: 0.5823405	best: 0.5823405 (1)	total: 151ms	remaining: 1m 15s
2:	learn: 0.5923537	test: 0.5727880	best: 0.5727880 (2)	total: 152ms	remaining: 50.4s
3:	learn: 0.5815174	test: 0.5633909	best: 0.5633909 (3)	total: 153ms	remaining: 38s
4:	learn: 0.5717567	test: 0.5544666	best: 0.5544666 (4)	total: 153ms	remaining: 30.5s
5:	learn: 0.5623455	test: 0.5455670	best: 0.5455670 (5)	total: 154ms	remaining: 25.5s
6:	learn: 0.5531780	test: 0.5370416	best: 0.5370416 (6)	total: 156ms	remaining: 22.1s
7:	learn: 0.5442129	test: 0.5289181	best: 0.5289181 (7)	total: 157ms	remaining: 19.4s
8:	learn: 0.5365000	test: 0.5216043	best: 0.5216043 (8)	total: 158ms	remaining: 17.4s
9:	learn: 0.5283225	test: 0.5141535	best: 0.5141535 (9)	total: 158ms	remaining: 15.7s
10:	learn: 0.5206842	test: 0.5071509	best: 0.5071509 (10)	total: 159ms	remaining: 14.3s
11:	learn: 0.5141503	test: 0.5006082	best: 0.5006082

175:	learn: 0.3394315	test: 0.3489099	best: 0.3488430 (173)	total: 290ms	remaining: 1.36s
176:	learn: 0.3391208	test: 0.3484081	best: 0.3484081 (176)	total: 291ms	remaining: 1.35s
177:	learn: 0.3389443	test: 0.3484161	best: 0.3484081 (176)	total: 292ms	remaining: 1.35s
178:	learn: 0.3388653	test: 0.3483529	best: 0.3483529 (178)	total: 293ms	remaining: 1.34s
179:	learn: 0.3386648	test: 0.3479957	best: 0.3479957 (179)	total: 294ms	remaining: 1.34s
180:	learn: 0.3385101	test: 0.3479096	best: 0.3479096 (180)	total: 295ms	remaining: 1.33s
181:	learn: 0.3383645	test: 0.3480931	best: 0.3479096 (180)	total: 296ms	remaining: 1.33s
182:	learn: 0.3382868	test: 0.3480945	best: 0.3479096 (180)	total: 297ms	remaining: 1.32s
183:	learn: 0.3380327	test: 0.3481765	best: 0.3479096 (180)	total: 298ms	remaining: 1.32s
184:	learn: 0.3379352	test: 0.3479772	best: 0.3479096 (180)	total: 298ms	remaining: 1.31s
185:	learn: 0.3378070	test: 0.3481088	best: 0.3479096 (180)	total: 299ms	remaining: 1.31s
186:	learn

321:	learn: 0.3185825	test: 0.3471526	best: 0.3467172 (252)	total: 422ms	remaining: 889ms
322:	learn: 0.3184481	test: 0.3471983	best: 0.3467172 (252)	total: 423ms	remaining: 886ms
323:	learn: 0.3182690	test: 0.3471830	best: 0.3467172 (252)	total: 424ms	remaining: 884ms
324:	learn: 0.3180810	test: 0.3472373	best: 0.3467172 (252)	total: 425ms	remaining: 882ms
325:	learn: 0.3177800	test: 0.3471136	best: 0.3467172 (252)	total: 426ms	remaining: 880ms
326:	learn: 0.3176622	test: 0.3470294	best: 0.3467172 (252)	total: 427ms	remaining: 878ms
327:	learn: 0.3174966	test: 0.3470398	best: 0.3467172 (252)	total: 427ms	remaining: 875ms
328:	learn: 0.3173766	test: 0.3470451	best: 0.3467172 (252)	total: 428ms	remaining: 873ms
329:	learn: 0.3171538	test: 0.3471313	best: 0.3467172 (252)	total: 429ms	remaining: 871ms
330:	learn: 0.3170976	test: 0.3470266	best: 0.3467172 (252)	total: 430ms	remaining: 868ms
331:	learn: 0.3170168	test: 0.3468359	best: 0.3467172 (252)	total: 431ms	remaining: 866ms
332:	learn

482:	learn: 0.2937717	test: 0.3423553	best: 0.3421052 (437)	total: 570ms	remaining: 610ms
483:	learn: 0.2935922	test: 0.3423825	best: 0.3421052 (437)	total: 571ms	remaining: 609ms
484:	learn: 0.2933984	test: 0.3423057	best: 0.3421052 (437)	total: 573ms	remaining: 608ms
485:	learn: 0.2932818	test: 0.3423584	best: 0.3421052 (437)	total: 574ms	remaining: 607ms
486:	learn: 0.2931502	test: 0.3424413	best: 0.3421052 (437)	total: 575ms	remaining: 605ms
487:	learn: 0.2929957	test: 0.3425044	best: 0.3421052 (437)	total: 576ms	remaining: 604ms
488:	learn: 0.2928585	test: 0.3424505	best: 0.3421052 (437)	total: 577ms	remaining: 602ms
489:	learn: 0.2927770	test: 0.3424918	best: 0.3421052 (437)	total: 577ms	remaining: 601ms
490:	learn: 0.2926758	test: 0.3423302	best: 0.3421052 (437)	total: 578ms	remaining: 600ms
491:	learn: 0.2925493	test: 0.3422987	best: 0.3421052 (437)	total: 579ms	remaining: 598ms
492:	learn: 0.2924006	test: 0.3422644	best: 0.3421052 (437)	total: 580ms	remaining: 597ms
493:	learn

...

720:	learn: 0.1526954	test: 0.3547728	best: 0.3441798 (116)	total: 8.53s	remaining: 3.3s
721:	learn: 0.1525885	test: 0.3548253	best: 0.3441798 (116)	total: 8.54s	remaining: 3.29s
722:	learn: 0.1524387	test: 0.3548300	best: 0.3441798 (116)	total: 8.56s	remaining: 3.28s
723:	learn: 0.1523863	test: 0.3548014	best: 0.3441798 (116)	total: 8.57s	remaining: 3.27s
724:	learn: 0.1523645	test: 0.3547997	best: 0.3441798 (116)	total: 8.59s	remaining: 3.26s
725:	learn: 0.1523434	test: 0.3548079	best: 0.3441798 (116)	total: 8.61s	remaining: 3.25s
726:	learn: 0.1523230	test: 0.3548063	best: 0.3441798 (116)	total: 8.62s	remaining: 3.24s
727:	learn: 0.1522671	test: 0.3547817	best: 0.3441798 (116)	total: 8.63s	remaining: 3.22s
728:	learn: 0.1521461	test: 0.3548157	best: 0.3441798 (116)	total: 8.64s	remaining: 3.21s
729:	learn: 0.1520381	test: 0.3548794	best: 0.3441798 (116)	total: 8.66s	remaining: 3.2s
730:	learn: 0.1518716	test: 0.3547583	best: 0.3441798 (116)	total: 8.67s	remaining: 3.19s
731:	learn: 

822:	learn: 0.1451949	test: 0.3551087	best: 0.3441798 (116)	total: 9.82s	remaining: 2.11s
823:	learn: 0.1451477	test: 0.3550777	best: 0.3441798 (116)	total: 9.83s	remaining: 2.1s
824:	learn: 0.1450724	test: 0.3551020	best: 0.3441798 (116)	total: 9.85s	remaining: 2.09s
825:	learn: 0.1450140	test: 0.3551232	best: 0.3441798 (116)	total: 9.86s	remaining: 2.08s
826:	learn: 0.1448668	test: 0.3551115	best: 0.3441798 (116)	total: 9.87s	remaining: 2.06s
827:	learn: 0.1447761	test: 0.3551922	best: 0.3441798 (116)	total: 9.88s	remaining: 2.05s
828:	learn: 0.1447650	test: 0.3552303	best: 0.3441798 (116)	total: 9.9s	remaining: 2.04s
829:	learn: 0.1447513	test: 0.3552285	best: 0.3441798 (116)	total: 9.91s	remaining: 2.03s
830:	learn: 0.1446663	test: 0.3552557	best: 0.3441798 (116)	total: 9.92s	remaining: 2.02s
831:	learn: 0.1444931	test: 0.3552608	best: 0.3441798 (116)	total: 9.93s	remaining: 2.01s
832:	learn: 0.1444201	test: 0.3553051	best: 0.3441798 (116)	total: 9.95s	remaining: 1.99s
833:	learn: 

919:	learn: 0.1388153	test: 0.3559768	best: 0.3441798 (116)	total: 11.1s	remaining: 965ms
920:	learn: 0.1386982	test: 0.3559629	best: 0.3441798 (116)	total: 11.1s	remaining: 953ms
921:	learn: 0.1386525	test: 0.3560522	best: 0.3441798 (116)	total: 11.1s	remaining: 941ms
922:	learn: 0.1385792	test: 0.3561008	best: 0.3441798 (116)	total: 11.1s	remaining: 929ms
923:	learn: 0.1385182	test: 0.3560777	best: 0.3441798 (116)	total: 11.1s	remaining: 917ms
924:	learn: 0.1384481	test: 0.3560951	best: 0.3441798 (116)	total: 11.2s	remaining: 905ms
925:	learn: 0.1384192	test: 0.3560795	best: 0.3441798 (116)	total: 11.2s	remaining: 893ms
926:	learn: 0.1383908	test: 0.3561105	best: 0.3441798 (116)	total: 11.2s	remaining: 880ms
927:	learn: 0.1383216	test: 0.3561046	best: 0.3441798 (116)	total: 11.2s	remaining: 868ms
928:	learn: 0.1382975	test: 0.3561072	best: 0.3441798 (116)	total: 11.2s	remaining: 856ms
929:	learn: 0.1382628	test: 0.3560822	best: 0.3441798 (116)	total: 11.2s	remaining: 844ms
930:	learn

324:	learn: 0.2243356	test: 0.3657374	best: 0.3580093 (87)
325:	learn: 0.2239820	test: 0.3657828	best: 0.3580093 (87)
326:	learn: 0.2237104	test: 0.3659750	best: 0.3580093 (87)
327:	learn: 0.2234377	test: 0.3659909	best: 0.3580093 (87)
328:	learn: 0.2231608	test: 0.3660639	best: 0.3580093 (87)
329:	learn: 0.2229861	test: 0.3661856	best: 0.3580093 (87)
330:	learn: 0.2227477	test: 0.3662668	best: 0.3580093 (87)
331:	learn: 0.2222833	test: 0.3661800	best: 0.3580093 (87)
332:	learn: 0.2220763	test: 0.3661697	best: 0.3580093 (87)
333:	learn: 0.2218473	test: 0.3661990	best: 0.3580093 (87)
334:	learn: 0.2216591	test: 0.3663211	best: 0.3580093 (87)
335:	learn: 0.2214678	test: 0.3663904	best: 0.3580093 (87)
336:	learn: 0.2212066	test: 0.3662789	best: 0.3580093 (87)
337:	learn: 0.2209902	test: 0.3662762	best: 0.3580093 (87)
338:	learn: 0.2207754	test: 0.3662959	best: 0.3580093 (87)
339:	learn: 0.2205512	test: 0.3662598	best: 0.3580093 (87)
340:	learn: 0.2202849	test: 0.3663280	best: 0.3580093 (8

915:	learn: 0.1474407	test: 0.3905131	best: 0.3580093 (87)
916:	learn: 0.1473683	test: 0.3905099	best: 0.3580093 (87)
917:	learn: 0.1472921	test: 0.3904546	best: 0.3580093 (87)
918:	learn: 0.1471881	test: 0.3904591	best: 0.3580093 (87)
919:	learn: 0.1471449	test: 0.3905170	best: 0.3580093 (87)
920:	learn: 0.1470040	test: 0.3905910	best: 0.3580093 (87)
921:	learn: 0.1469648	test: 0.3907258	best: 0.3580093 (87)
922:	learn: 0.1468870	test: 0.3908049	best: 0.3580093 (87)
923:	learn: 0.1468023	test: 0.3908312	best: 0.3580093 (87)
924:	learn: 0.1467313	test: 0.3908537	best: 0.3580093 (87)
925:	learn: 0.1466065	test: 0.3909381	best: 0.3580093 (87)
926:	learn: 0.1465369	test: 0.3909626	best: 0.3580093 (87)
927:	learn: 0.1464203	test: 0.3910746	best: 0.3580093 (87)
928:	learn: 0.1463571	test: 0.3910677	best: 0.3580093 (87)
929:	learn: 0.1462782	test: 0.3910965	best: 0.3580093 (87)
930:	learn: 0.1461937	test: 0.3910726	best: 0.3580093 (87)
931:	learn: 0.1461546	test: 0.3910670	best: 0.3580093 (8